# 🦙 Fully Open-Source RAG pipeline using LlamaIndex + FAISS + local LLM


This script cannot be run in colab, as ollama server could not be install in colab

In [1]:
pip install llama-index-core llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-vector-stores-faiss


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
# Install dependencies (run in terminal once)

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
! ollama pull llama3

In [ ]:
# 🦙 Fully Open-Source RAG pipeline using LlamaIndex + FAISS + local LLM


# 1️⃣ Load your custom data (directory of .txt, .pdf, .md, etc.)
data_path = "path_to_your_documents"
documents = SimpleDirectoryReader(data_path).load_data()


# 2️⃣ Create a local embedding model (Hugging Face)
# Choose any small embedding model that works offline
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


# 3️⃣ Setup FAISS vector store (fully local)
faiss_store = FaissVectorStore.from_params(dim=384)  # dimension must match embedding size
storage_context = StorageContext.from_defaults(vector_store=faiss_store)


# 4️⃣ Create an index (store embeddings locally)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)


# 5️⃣ Configure your local LLM (via Ollama)
# Make sure Ollama is installed and running (https://ollama.ai)
# You can use: `ollama pull llama3`  or  `ollama pull mistral`
llm = Ollama(model="llama3")  # or "mistral", "phi3", etc.


# 6️⃣ Create the query engine (retrieve + reason locally)
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=3  # number of most relevant chunks to retrieve
)


# 7️⃣ Function to query your data locally
def generate_response(query: str):
    """RAG query using local embeddings + local LLM."""
    response = query_engine.query(query)
    return str(response)


# Example usage
if __name__ == "__main__":
    user_query = "How do I reset my device?"
    print(generate_response(user_query))
